# Cleanup and Exploration of 2019 AQ Data

In [1]:
import pandas as pd
import json as json

In [2]:
df = pd.read_csv('2019 AQ.csv')

In [3]:
df.head()

,date,parameter,location,value,unit,city,attribution,averagingperiod,coordinates,country,sourcename,sourcetype,mobile
0,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",o3,Ashton,0.021,ppm,Walterboro,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.0078, longitude=-80.965}",US,AirNow,government,False
1,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",pm25,Ashton,3.000,µg/m³,Walterboro,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.0078, longitude=-80.965}",US,AirNow,government,False
2,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",o3,Congaree Bluff,0.026,ppm,Columbia,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.8147, longitude=-80.7811}",US,AirNow,government,False
3,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",so2,Congaree Bluff,0.000,ppm,Columbia,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.8147, longitude=-80.7811}",US,AirNow,government,False
4,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",pm25,TK Gregg,1.200,µg/m³,Spartanburg,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=34.9556, longitude=-81.9248}",US,AirNow,government,False


The entry is not valid json.  The field will need processing code.

In [4]:
def validate_json_fields(field: str) -> bool:
    """This function validates a JSON entry is valid.  Returns True if so, False if otherwise."""
    try:
        json.loads(field)
        return True
    except ValueError:
        return False

In [5]:
def validate_json_column(series: pd.Series):
    temp_df = pd.DataFrame()
    temp_df['values'] = series
    temp_df['validity'] = series.apply(validate_json_fields)
    return temp_df

## Validation of composite fields as json

In [6]:
output = validate_json_column(df['date'])

In [7]:
output.head()

,values,validity
0,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",False
1,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",False
2,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",False
3,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",False
4,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",False


In [8]:
output[output['validity'] == True].describe()

,values,validity
count,0,0
unique,0,0
top,NaN,NaN
freq,NaN,NaN


In [9]:
output = validate_json_column(df['attribution'])
output.head()

,values,validity
0,"[{name=US EPA AirNow, url=http://www.airnow.go...",False
1,"[{name=US EPA AirNow, url=http://www.airnow.go...",False
2,"[{name=US EPA AirNow, url=http://www.airnow.go...",False
3,"[{name=US EPA AirNow, url=http://www.airnow.go...",False
4,"[{name=US EPA AirNow, url=http://www.airnow.go...",False


In [10]:
output[output['validity'] == True].describe()

,values,validity
count,0,0
unique,0,0
top,NaN,NaN
freq,NaN,NaN


In [11]:
output = validate_json_column(df['averagingperiod'])
output.head()

,values,validity
0,"{unit=hours, value=1.0}",False
1,"{unit=hours, value=1.0}",False
2,"{unit=hours, value=1.0}",False
3,"{unit=hours, value=1.0}",False
4,"{unit=hours, value=1.0}",False


In [12]:
output[output['validity'] == True].describe()

,values,validity
count,0,0
unique,0,0
top,NaN,NaN
freq,NaN,NaN


In [13]:
output = validate_json_column(df['coordinates'])
output.head()

,values,validity
0,"{latitude=33.0078, longitude=-80.965}",False
1,"{latitude=33.0078, longitude=-80.965}",False
2,"{latitude=33.8147, longitude=-80.7811}",False
3,"{latitude=33.8147, longitude=-80.7811}",False
4,"{latitude=34.9556, longitude=-81.9248}",False


In [14]:
output[output['validity'] == True].describe()

,values,validity
count,0,0
unique,0,0
top,NaN,NaN
freq,NaN,NaN


All of the composite fields are in a "JSON-like format"  need to convert to JSON.

## Reprocessing the Multivalue Fields into Valid JSON

In [15]:
def correct_field_to_json(arg: str) -> str:
    string = arg.lstrip("{{")
    string = string.rstrip("]}")
    strings = string.split(",")
    new_strings = []
    for entry in strings:
        index = entry.find('=')
        entry = '"' + entry[0:index] + '"' + ':' + '"'+ entry[index+1:len(entry)]+'"'
        new_strings.append(entry)
    output_string = (',').join(entry for entry in new_strings)
    output_string = '{' + output_string + '}'
    return output_string

In [16]:
def correct_json_column(series: pd.Series, column_name: str) -> pd.DataFrame:
    temp_df = pd.DataFrame()
    temp_df[column_name] = series.apply(correct_field_to_json)
    return temp_df

In [17]:
test = correct_json_column(df['date'], 'test date conversion')
results = validate_json_column(test['test date conversion'])
results[results['validity'] == False].describe()

,values,validity
count,0,0
unique,0,0
top,NaN,NaN
freq,NaN,NaN


In [18]:
df['cleaneddate'] = test

In [19]:
test = correct_json_column(df['attribution'], 'test attribute conversion')
results = validate_json_column(test['test attribute conversion'])
results[results['validity'] == False].describe()

,values,validity
count,0,0
unique,0,0
top,NaN,NaN
freq,NaN,NaN


In [20]:
results.head()

,values,validity
0,"{""[{name"":""US EPA AirNow"","" url"":""http://www.a...",True
1,"{""[{name"":""US EPA AirNow"","" url"":""http://www.a...",True
2,"{""[{name"":""US EPA AirNow"","" url"":""http://www.a...",True
3,"{""[{name"":""US EPA AirNow"","" url"":""http://www.a...",True
4,"{""[{name"":""US EPA AirNow"","" url"":""http://www.a...",True


In [21]:
df['cleanedattribution'] = test

In [22]:
test = correct_json_column(df['averagingperiod'], 'test averaging period conversion')
results = validate_json_column(test['test averaging period conversion'])
results[results['validity'] == False].describe()

,values,validity
count,0,0
unique,0,0
top,NaN,NaN
freq,NaN,NaN


In [23]:
df['cleanedaveragingperiod'] = test

In [24]:
test = correct_json_column(df['coordinates'], 'test coordinate conversion')
results = validate_json_column(test['test coordinate conversion'])
results[results['validity'] == False].describe()

,values,validity
count,0,0
unique,0,0
top,NaN,NaN
freq,NaN,NaN


In [25]:
df['cleanedcoordinates'] = test

In [26]:
df.head()

,date,parameter,location,value,unit,city,attribution,averagingperiod,coordinates,country,sourcename,sourcetype,mobile,cleaneddate,cleanedattribution,cleanedaveragingperiod,cleanedcoordinates
0,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",o3,Ashton,0.021,ppm,Walterboro,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.0078, longitude=-80.965}",US,AirNow,government,False,"{""utc"":""2019-01-02T18:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.0078"","" longitude"":""-80.965""}"
1,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",pm25,Ashton,3.000,µg/m³,Walterboro,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.0078, longitude=-80.965}",US,AirNow,government,False,"{""utc"":""2019-01-02T18:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.0078"","" longitude"":""-80.965""}"
2,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",o3,Congaree Bluff,0.026,ppm,Columbia,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.8147, longitude=-80.7811}",US,AirNow,government,False,"{""utc"":""2019-01-02T18:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.8147"","" longitude"":""-80.7811""}"
3,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",so2,Congaree Bluff,0.000,ppm,Columbia,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.8147, longitude=-80.7811}",US,AirNow,government,False,"{""utc"":""2019-01-02T18:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.8147"","" longitude"":""-80.7811""}"
4,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",pm25,TK Gregg,1.200,µg/m³,Spartanburg,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=34.9556, longitude=-81.9248}",US,AirNow,government,False,"{""utc"":""2019-01-02T18:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""34.9556"","" longitude"":""-81.9248""}"


In [27]:
import ujson as ujson

In [28]:
#df_trans = pd.concat([df_trans, df_trans.cleanedcoordinates.apply(json.loads).apply(pd.Series)], axis = 1)
expanded_datascope = df['cleanedcoordinates'].apply(ujson.loads)
expanded_datascope = pd.json_normalize(expanded_datascope)
df = pd.concat([df, expanded_datascope], axis =1)

In [29]:
df.head()

,date,parameter,location,value,unit,city,attribution,averagingperiod,coordinates,country,sourcename,sourcetype,mobile,cleaneddate,cleanedattribution,cleanedaveragingperiod,cleanedcoordinates,latitude,longitude
0,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",o3,Ashton,0.021,ppm,Walterboro,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.0078, longitude=-80.965}",US,AirNow,government,False,"{""utc"":""2019-01-02T18:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.0078"","" longitude"":""-80.965""}",33.0078,-80.965
1,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",pm25,Ashton,3.000,µg/m³,Walterboro,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.0078, longitude=-80.965}",US,AirNow,government,False,"{""utc"":""2019-01-02T18:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.0078"","" longitude"":""-80.965""}",33.0078,-80.965
2,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",o3,Congaree Bluff,0.026,ppm,Columbia,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.8147, longitude=-80.7811}",US,AirNow,government,False,"{""utc"":""2019-01-02T18:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.8147"","" longitude"":""-80.7811""}",33.8147,-80.7811
3,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",so2,Congaree Bluff,0.000,ppm,Columbia,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.8147, longitude=-80.7811}",US,AirNow,government,False,"{""utc"":""2019-01-02T18:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.8147"","" longitude"":""-80.7811""}",33.8147,-80.7811
4,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",pm25,TK Gregg,1.200,µg/m³,Spartanburg,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=34.9556, longitude=-81.9248}",US,AirNow,government,False,"{""utc"":""2019-01-02T18:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""34.9556"","" longitude"":""-81.9248""}",34.9556,-81.9248


In [30]:
#df_trans = pd.concat([df_trans, df_trans.cleaneddate.apply(ujson.loads).apply(pd.Series)], axis = 1)
expanded_datascope = df['cleaneddate'].apply(ujson.loads)
expanded_datascope = pd.json_normalize(expanded_datascope)
df = pd.concat([df, expanded_datascope], axis =1)

In [31]:
df.head()

,date,parameter,location,value,unit,city,attribution,averagingperiod,coordinates,country,...,sourcetype,mobile,cleaneddate,cleanedattribution,cleanedaveragingperiod,cleanedcoordinates,latitude,longitude,utc,local
0,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",o3,Ashton,0.021,ppm,Walterboro,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.0078, longitude=-80.965}",US,...,government,False,"{""utc"":""2019-01-02T18:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.0078"","" longitude"":""-80.965""}",33.0078,-80.965,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00
1,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",pm25,Ashton,3.000,µg/m³,Walterboro,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.0078, longitude=-80.965}",US,...,government,False,"{""utc"":""2019-01-02T18:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.0078"","" longitude"":""-80.965""}",33.0078,-80.965,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00
2,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",o3,Congaree Bluff,0.026,ppm,Columbia,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.8147, longitude=-80.7811}",US,...,government,False,"{""utc"":""2019-01-02T18:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.8147"","" longitude"":""-80.7811""}",33.8147,-80.7811,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00
3,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",so2,Congaree Bluff,0.000,ppm,Columbia,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.8147, longitude=-80.7811}",US,...,government,False,"{""utc"":""2019-01-02T18:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.8147"","" longitude"":""-80.7811""}",33.8147,-80.7811,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00
4,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",pm25,TK Gregg,1.200,µg/m³,Spartanburg,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=34.9556, longitude=-81.9248}",US,...,government,False,"{""utc"":""2019-01-02T18:00:00.000Z"","" local"":""20...","{""[{name"":""US EPA AirNow"","" url"":""http://www.a...","{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""34.9556"","" longitude"":""-81.9248""}",34.9556,-81.9248,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00


In [32]:
#df_trans = pd.concat([df_trans, df_trans.cleanedattribution.apply(json.loads).apply(pd.Series)], axis = 1) This line is EXTREMELY SLOW TRYING TO OPTIMIZE.
expanded_datascope = df['cleanedattribution'].apply(ujson.loads)
expanded_datascope = pd.json_normalize(expanded_datascope)
df = pd.concat([df, expanded_datascope], axis =1)

In [33]:
df.head()

,date,parameter,location,value,unit,city,attribution,averagingperiod,coordinates,country,...,cleanedcoordinates,latitude,longitude,utc,local,[{name,url,{name,Great Lake,and Energ
0,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",o3,Ashton,0.021,ppm,Walterboro,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.0078, longitude=-80.965}",US,...,"{""latitude"":""33.0078"","" longitude"":""-80.965""}",33.0078,-80.965,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,South Carolina DHEC,NaN,NaN
1,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",pm25,Ashton,3.000,µg/m³,Walterboro,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.0078, longitude=-80.965}",US,...,"{""latitude"":""33.0078"","" longitude"":""-80.965""}",33.0078,-80.965,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,South Carolina DHEC,NaN,NaN
2,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",o3,Congaree Bluff,0.026,ppm,Columbia,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.8147, longitude=-80.7811}",US,...,"{""latitude"":""33.8147"","" longitude"":""-80.7811""}",33.8147,-80.7811,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,South Carolina DHEC,NaN,NaN
3,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",so2,Congaree Bluff,0.000,ppm,Columbia,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.8147, longitude=-80.7811}",US,...,"{""latitude"":""33.8147"","" longitude"":""-80.7811""}",33.8147,-80.7811,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,South Carolina DHEC,NaN,NaN
4,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",pm25,TK Gregg,1.200,µg/m³,Spartanburg,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=34.9556, longitude=-81.9248}",US,...,"{""latitude"":""34.9556"","" longitude"":""-81.9248""}",34.9556,-81.9248,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,South Carolina DHEC,NaN,NaN


In [34]:
df.drop(' {name', axis=1, inplace=True)

In [35]:
df.head()

,date,parameter,location,value,unit,city,attribution,averagingperiod,coordinates,country,...,cleanedaveragingperiod,cleanedcoordinates,latitude,longitude,utc,local,[{name,url,Great Lake,and Energ
0,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",o3,Ashton,0.021,ppm,Walterboro,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.0078, longitude=-80.965}",US,...,"{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.0078"","" longitude"":""-80.965""}",33.0078,-80.965,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN
1,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",pm25,Ashton,3.000,µg/m³,Walterboro,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.0078, longitude=-80.965}",US,...,"{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.0078"","" longitude"":""-80.965""}",33.0078,-80.965,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN
2,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",o3,Congaree Bluff,0.026,ppm,Columbia,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.8147, longitude=-80.7811}",US,...,"{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.8147"","" longitude"":""-80.7811""}",33.8147,-80.7811,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN
3,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",so2,Congaree Bluff,0.000,ppm,Columbia,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.8147, longitude=-80.7811}",US,...,"{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""33.8147"","" longitude"":""-80.7811""}",33.8147,-80.7811,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN
4,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",pm25,TK Gregg,1.200,µg/m³,Spartanburg,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=34.9556, longitude=-81.9248}",US,...,"{""unit"":""hours"","" value"":""1.0""}","{""latitude"":""34.9556"","" longitude"":""-81.9248""}",34.9556,-81.9248,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN


In [36]:
#df_trans = pd.concat([df_trans, df_trans.cleanedaveragingperiod.apply(json.loads).apply(pd.Series)], axis = 1)
expanded_datascope = df['cleanedaveragingperiod'].apply(ujson.loads)
expanded_datascope = pd.json_normalize(expanded_datascope)
df = pd.concat([df, expanded_datascope], axis =1)

In [37]:
df.head()

,date,parameter,location,value,unit,city,attribution,averagingperiod,coordinates,country,...,latitude,longitude,utc,local,[{name,url,Great Lake,and Energ,unit,value
0,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",o3,Ashton,0.021,ppm,Walterboro,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.0078, longitude=-80.965}",US,...,33.0078,-80.965,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0
1,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",pm25,Ashton,3.000,µg/m³,Walterboro,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.0078, longitude=-80.965}",US,...,33.0078,-80.965,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0
2,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",o3,Congaree Bluff,0.026,ppm,Columbia,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.8147, longitude=-80.7811}",US,...,33.8147,-80.7811,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0
3,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",so2,Congaree Bluff,0.000,ppm,Columbia,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.8147, longitude=-80.7811}",US,...,33.8147,-80.7811,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0
4,"{utc=2019-01-02T18:00:00.000Z, local=2019-01-0...",pm25,TK Gregg,1.200,µg/m³,Spartanburg,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=34.9556, longitude=-81.9248}",US,...,34.9556,-81.9248,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0


In [38]:
df.drop('date', axis =1, inplace=True)

In [39]:
df.head()

,parameter,location,value,unit,city,attribution,averagingperiod,coordinates,country,sourcename,...,latitude,longitude,utc,local,[{name,url,Great Lake,and Energ,unit,value
0,o3,Ashton,0.021,ppm,Walterboro,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.0078, longitude=-80.965}",US,AirNow,...,33.0078,-80.965,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0
1,pm25,Ashton,3.000,µg/m³,Walterboro,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.0078, longitude=-80.965}",US,AirNow,...,33.0078,-80.965,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0
2,o3,Congaree Bluff,0.026,ppm,Columbia,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.8147, longitude=-80.7811}",US,AirNow,...,33.8147,-80.7811,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0
3,so2,Congaree Bluff,0.000,ppm,Columbia,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=33.8147, longitude=-80.7811}",US,AirNow,...,33.8147,-80.7811,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0
4,pm25,TK Gregg,1.200,µg/m³,Spartanburg,"[{name=US EPA AirNow, url=http://www.airnow.go...","{unit=hours, value=1.0}","{latitude=34.9556, longitude=-81.9248}",US,AirNow,...,34.9556,-81.9248,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0


In [40]:
df.drop(['attribution','averagingperiod','coordinates','cleanedaveragingperiod','cleanedcoordinates'], axis = 1, inplace = True)

In [41]:
df.head()

,parameter,location,value,unit,city,country,sourcename,sourcetype,mobile,cleaneddate,...,latitude,longitude,utc,local,[{name,url,Great Lake,and Energ,unit,value
0,o3,Ashton,0.021,ppm,Walterboro,US,AirNow,government,False,"{""utc"":""2019-01-02T18:00:00.000Z"","" local"":""20...",...,33.0078,-80.965,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0
1,pm25,Ashton,3.000,µg/m³,Walterboro,US,AirNow,government,False,"{""utc"":""2019-01-02T18:00:00.000Z"","" local"":""20...",...,33.0078,-80.965,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0
2,o3,Congaree Bluff,0.026,ppm,Columbia,US,AirNow,government,False,"{""utc"":""2019-01-02T18:00:00.000Z"","" local"":""20...",...,33.8147,-80.7811,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0
3,so2,Congaree Bluff,0.000,ppm,Columbia,US,AirNow,government,False,"{""utc"":""2019-01-02T18:00:00.000Z"","" local"":""20...",...,33.8147,-80.7811,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0
4,pm25,TK Gregg,1.200,µg/m³,Spartanburg,US,AirNow,government,False,"{""utc"":""2019-01-02T18:00:00.000Z"","" local"":""20...",...,34.9556,-81.9248,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0


In [42]:
df.drop(['cleaneddate','cleanedattribution'], axis = 1, inplace = True)

In [43]:
df.head()

,parameter,location,value,unit,city,country,sourcename,sourcetype,mobile,latitude,longitude,utc,local,[{name,url,Great Lake,and Energ,unit,value
0,o3,Ashton,0.021,ppm,Walterboro,US,AirNow,government,False,33.0078,-80.965,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0
1,pm25,Ashton,3.000,µg/m³,Walterboro,US,AirNow,government,False,33.0078,-80.965,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0
2,o3,Congaree Bluff,0.026,ppm,Columbia,US,AirNow,government,False,33.8147,-80.7811,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0
3,so2,Congaree Bluff,0.000,ppm,Columbia,US,AirNow,government,False,33.8147,-80.7811,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0
4,pm25,TK Gregg,1.200,µg/m³,Spartanburg,US,AirNow,government,False,34.9556,-81.9248,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0


In [44]:
#df_trans.rename(columns={df_trans.columns[15]: 'averaging time unit'}, inplace=True)
df.columns.values[3]='concentration unit'

In [45]:
df.head()

,parameter,location,value,concentration unit,city,country,sourcename,sourcetype,mobile,latitude,longitude,utc,local,[{name,url,Great Lake,and Energ,unit,value
0,o3,Ashton,0.021,ppm,Walterboro,US,AirNow,government,False,33.0078,-80.965,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0
1,pm25,Ashton,3.000,µg/m³,Walterboro,US,AirNow,government,False,33.0078,-80.965,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0
2,o3,Congaree Bluff,0.026,ppm,Columbia,US,AirNow,government,False,33.8147,-80.7811,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0
3,so2,Congaree Bluff,0.000,ppm,Columbia,US,AirNow,government,False,33.8147,-80.7811,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0
4,pm25,TK Gregg,1.200,µg/m³,Spartanburg,US,AirNow,government,False,34.9556,-81.9248,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0


In [46]:
#df_trans.rename(columns={df_trans.columns[15]: 'averaging time unit'}, inplace=True)
df.columns.values[15] = 'averaging time unit'

In [47]:
#df_trans.rename(columns={df_trans.columns[13]: 'source'}, inplace=True)
df.columns.values[13] = 'source'

In [48]:
df.head()

,parameter,location,value,concentration unit,city,country,sourcename,sourcetype,mobile,latitude,longitude,utc,local,source,url,averaging time unit,and Energ,unit,value
0,o3,Ashton,0.021,ppm,Walterboro,US,AirNow,government,False,33.0078,-80.965,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0
1,pm25,Ashton,3.000,µg/m³,Walterboro,US,AirNow,government,False,33.0078,-80.965,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0
2,o3,Congaree Bluff,0.026,ppm,Columbia,US,AirNow,government,False,33.8147,-80.7811,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0
3,so2,Congaree Bluff,0.000,ppm,Columbia,US,AirNow,government,False,33.8147,-80.7811,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0
4,pm25,TK Gregg,1.200,µg/m³,Spartanburg,US,AirNow,government,False,34.9556,-81.9248,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0


In [49]:
#df_trans.rename(columns={df_trans.columns[16]: 'averaging time'},inplace=True)
df.columns.values[16] = 'averaging time'

In [50]:
df.head()

,parameter,location,value,concentration unit,city,country,sourcename,sourcetype,mobile,latitude,longitude,utc,local,source,url,averaging time unit,averaging time,unit,value
0,o3,Ashton,0.021,ppm,Walterboro,US,AirNow,government,False,33.0078,-80.965,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0
1,pm25,Ashton,3.000,µg/m³,Walterboro,US,AirNow,government,False,33.0078,-80.965,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0
2,o3,Congaree Bluff,0.026,ppm,Columbia,US,AirNow,government,False,33.8147,-80.7811,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0
3,so2,Congaree Bluff,0.000,ppm,Columbia,US,AirNow,government,False,33.8147,-80.7811,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0
4,pm25,TK Gregg,1.200,µg/m³,Spartanburg,US,AirNow,government,False,34.9556,-81.9248,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,NaN,NaN,hours,1.0


In [51]:
df.to_csv('2019 AQ Clean.csv', index=False)

In [53]:
df.drop(['averaging time unit', 'averaging time'], axis =1, inplace=True)

In [54]:
df.head()

,parameter,location,value,concentration unit,city,country,sourcename,sourcetype,mobile,latitude,longitude,utc,local,source,url,unit,value
0,o3,Ashton,0.021,ppm,Walterboro,US,AirNow,government,False,33.0078,-80.965,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,hours,1.0
1,pm25,Ashton,3.000,µg/m³,Walterboro,US,AirNow,government,False,33.0078,-80.965,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,hours,1.0
2,o3,Congaree Bluff,0.026,ppm,Columbia,US,AirNow,government,False,33.8147,-80.7811,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,hours,1.0
3,so2,Congaree Bluff,0.000,ppm,Columbia,US,AirNow,government,False,33.8147,-80.7811,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,hours,1.0
4,pm25,TK Gregg,1.200,µg/m³,Spartanburg,US,AirNow,government,False,34.9556,-81.9248,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,hours,1.0


In [55]:
df.columns.values[15] = 'averaging time unit'
df.columns.values[16] = 'averaging time'
df.head()

,parameter,location,value,concentration unit,city,country,sourcename,sourcetype,mobile,latitude,longitude,utc,local,source,url,averaging time unit,averaging time
0,o3,Ashton,0.021,ppm,Walterboro,US,AirNow,government,False,33.0078,-80.965,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,hours,1.0
1,pm25,Ashton,3.000,µg/m³,Walterboro,US,AirNow,government,False,33.0078,-80.965,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,hours,1.0
2,o3,Congaree Bluff,0.026,ppm,Columbia,US,AirNow,government,False,33.8147,-80.7811,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,hours,1.0
3,so2,Congaree Bluff,0.000,ppm,Columbia,US,AirNow,government,False,33.8147,-80.7811,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,hours,1.0
4,pm25,TK Gregg,1.200,µg/m³,Spartanburg,US,AirNow,government,False,34.9556,-81.9248,2019-01-02T18:00:00.000Z,2019-01-02T13:00:00-05:00,US EPA AirNow,null,hours,1.0


In [56]:
df.to_csv('2019 AQ Clean.csv', index=False)